# irodsConnector 

## Create an iRODS session (connection to iRODS server)

In [1]:
import json
import os
from getpass import getpass
from pathlib import Path

from ibridges import Session
from ibridges.path import IrodsPath


#### Define your irods_environment.json
iRODS clients expect the environment specifications to be placed in a folder `~/.irods`. this can look differently for linux/Mac and Windows. 

Please make sure you place the environment file as a plain json in the location below:

In [2]:
env_file = Path(os.path.expanduser("~")).joinpath(".irods", "irods_environment.json")
os.path.exists(env_file)

True

### Password authentication

In [3]:
with open(env_file, "r") as f:
    ienv = json.load(f)
password = getpass("Your iRODS password")
session = Session(irods_env=ienv, password=password)

Auth with password


### Cache your password

In [4]:
session.write_pam_password()

In [5]:
# Close the irods session when you do not need it any longer.
session.close()

Now you can use the next cell to login without providing your password. 
**Note** the cached password will expire.

### Authenticate using the cached password ~/.irods/.rodsA

In [7]:
session.has_valid_irods_session()
print(session._irods_session)

None


In [8]:
session = Session(irods_env_path=env_file)

Auth without password


### Checking some session parameters

In [9]:
print(session.username)
print(session.default_resc) # the resource to which data will be uploaded
print(session.zone) # default home for iRODS /zone/home/username
print(session.server_version)

r.d.schram@uu.nl
irodsResc
nluu12p
(4, 2, 12)


## Working with data

In [10]:
from ibridges.data_operations import create_collection, get_size, obj_replicas
from ibridges.data_operations import upload, download, get_dataobject, get_collection

### Create a collection

On a local file name we have folders/directories and files. In iRODS we call them collections and data objects. In contrast to files, data objects carry system metadata and user defined metadata. Likewise for collections.

In [11]:
# build the path to your home collection on the iRODS server
print(IrodsPath(session, "~"))

/nluu12p/home/research-test-christine


In [12]:
# extend the home path with a new sub collection
irods_path = IrodsPath(session, "~", "new_coll")

# assure that the collection does not already exist
print(irods_path.collection_exists())

# create the collection or simply retrieve the collection if it already existed 
coll = create_collection(session, irods_path)

False


Some information about the collection like name, path and data objects in the collection, cumulative size:

In [13]:
print('collection name\t', coll.name)
print('collection path\t', coll.path)
print('data objects\t', coll.data_objects)
print('cumulative size\t', get_size(session, coll))

collection name	 new_coll
collection path	 /nluu12p/home/research-test-christine/new_coll
data objects	 []
cumulative size	 0


### Upload a file or folder

In [14]:
# create a local file
local_path = Path.expanduser(Path('~')).joinpath("demofile.txt")
f = open(local_path, "a")
f.write("My content! Super important")
f.close()

# upload the file to our collection
upload(session, local_path, irods_path, overwrite = True)

# list again the data object in te collection
# print('data objects\t', coll.data_objects)
print('cumulative size\t', get_size(session, coll))

data objects	 [<iRODSDataObject 24476427 demofile.txt>]
cumulative size	 594


### Retrieve a  data object from iRODS and checking their status

In [15]:
obj_path = IrodsPath(session, coll.path, 'demofile.txt')
obj = get_dataobject(session, obj_path)

Some system metadata of the object:

In [16]:
print('data object name\t', obj.name)
print('data object path\t', obj.path)
print('data object size\t', obj.size)
print('data object checksum\t', obj.checksum)
print('data replicas\t', obj_replicas(obj))

data object name	 demofile.txt
data object path	 /nluu12p/home/research-test-christine/new_coll/demofile.txt
data object size	 594
data object checksum	 sha2:AMhDzkiw4UzqvaHoKyrHYvhpaae2MRTbKUYBKE4aCto=
data replicas	 [(0, 'lp0054_03', 'sha2:AMhDzkiw4UzqvaHoKyrHYvhpaae2MRTbKUYBKE4aCto=', 594, 'good')]


### Open a data object in read or write mode

In [17]:
stream = obj.open('r')
text = stream.read().decode()
stream.close()
print(text)

My content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super important


In [18]:
# overwrite the content of the data object
stream = obj.open('w')
stream.write('My new text'.encode())
stream.close()

In [19]:
# check the changes
with obj.open('r') as objRead:
    print(objRead.read().decode())

My new text


### Download a data object or collection

In [20]:
ipath = IrodsPath(session, coll.path)
ipath.parts

('/', 'nluu12p', 'home', 'research-test-christine', 'new_coll')

In [21]:
from pathlib import Path
local_path = Path(os.path.expanduser("~/Downloads"))
print("Download folder exists", local_path.is_dir())
download(session, ipath, local_path, overwrite=True)

Download folder exists True


### Delete a data object or collection

In [22]:
obj.unlink()
coll.remove()
ipath.remove()

## User defined metadata of data objects and collections

In [23]:
# upload a file
local_path = Path(os.path.expanduser("~")).joinpath("demofile.txt")
irods_path = IrodsPath(session, "~", "new_coll")
upload(session, local_path, irods_path, overwrite = True)

UNIX_FILE_CREATE_ERR: None

### Retrieve an iRODS object or collection and list its metadata

In [ ]:
obj = get_dataobject(session, irods_path.joinpath("demofile.txt")) 

In [ ]:
from ibridges.meta import MetaData
obj_meta = MetaData(obj)
print(obj_meta)

### View, add, set and delete metadata

In [ ]:
obj_meta.add('NewKey', 'NewValue')
obj_meta.add('NewKey', 'AnotherValue')
print(obj_meta)

In [ ]:
obj_meta.delete('NewKey', 'NewValue')
print(obj_meta)

We can also set the meta data to a single key, value, units pair. This will remove any other entries with the same key.

In [ ]:
obj_meta.set("NewKey", "YetAnotherValue")
print(obj_meta)

### Accessing metadata 
With the orint function you can quickly inspect the metadata of an iRODS collection or object. If you want to extract and do something with the metadata, use the `__iter__` function. We give a small example below where we assume that the metadata contains a key/name *AUTHOR*:

In [ ]:
for md in obj_meta.__iter__():
    print(md.name, "has value", md.value)

## Search for data

How can you retrieve the location of an iRODS collection or data object by its metadata or partial part? To this end we provide the function `search`

In [ ]:
from ibridges.search import search_data

#all collections and objects with 
key_vals = {"NEWKEY": "YetAnotherValue"}
result = search_data(session, key_vals=key_vals)
print(result)

The result gives us the collection, the object name, and its checksum.

The wildcard in iRODS searches is the `%`.

In [ ]:
result = search_data(session, path=session.home+"/new_coll/%")
print(result)

## Resources and handling resources

In [ ]:
from ibridges.resources import Resources
resources = Resources(session)

### Check if default resource exists

In [ ]:
default_resc = resources.get_resource(session._irods_env.get("irods_default_resource", ""))
print(default_resc.name)
print(default_resc.free_space) # Metadata how much bytes are left on resource, None if not set
print(default_resc.type) # Storage policy

### Listing resources

In [ ]:
print(resources.resources()) # all resources
print()
print(resources.root_resources) # all writeable resources (name, status, free space, context)

### Retrieve current free space
In contrast to `resc.free_space` the function `get_free_space` accumulates all free space in the subtree starting with the resource as parent.

In [ ]:
resources.get_free_space(session._irods_env.get("irods_default_resource", "")) # default resource name

## Tickets (access string to collection or data object)

### List all tickets which you issued

In [ ]:
from ibridges.tickets import Tickets
tickets = Tickets(session)
print(tickets.all_ticket_strings)
print(list(tickets)) # (ticket string, access mode, object or collection id, expiry date in epoche)

### Issue a ticket

In [ ]:
from datetime import datetime, timedelta

objPath = "/nluu12p/home/research-test-christine/books/BenHur.txt"
ticket = tickets.create_ticket(obj_path=objPath, ticket_type="write", 
                               expiry_date=datetime.today() + timedelta(days=1))  # allow write access

In [ ]:
print("Ticket strings:\t", tickets.all_ticket_strings)
for ticket in tickets:
    print(ticket)

### Fetch and delete a ticket

In [ ]:
ticket = tickets.get_ticket(tickets.all_ticket_strings[0])
tickets.delete_ticket(ticket)

## Rules
Execute an iRODS rule from a rule file:

In [ ]:
from ibridges.rules import execute_rule
rule_file = "tutorials/example_rules/example.r"
stdout, stderr = execute_rule(session, rule_file, {})

In [ ]:
print(stdout)
print(stderr)

### Overwrite parameters in iRODS rules
iRODS rule files end with a line like `input *in="This is a string or a path or etc"`. In this example there is an input parameter called `'*in'` and it takes the value `"This is a string or a path or etc"`. We can overwrite these values by passing a python dictionary:

In [ ]:
params = {'*in': '"Another input"'}
stdout, stderr = execute_rule(session, rule_file, params)
print(stdout)

Changing the type of the parameter from str to int, you can also see that keys in the dictionary which do not correspond to an input parameter, are simply ignored.

In [ ]:
params = {'*in': 4, '*another_val': '"Value"'}
stdout, stderr = execute_rule(session, rule_file, params)
print(stdout)

## Permissions

### Accessing the permissions of a data object or collection in iRODS

Objects and collections have permissions attached to them. Permissions, which work like access levels, must be specified per user or group. The basic permissions are `own` (implies reading and writing), `modify object` (editing and reading), and `read object`.

In [ ]:
from ibridges.permissions import Permissions

# select a file to inspect and set permissions on
item_path = session.home # Path to collection or data object
item = get_collection(session, item_path) # TODO: exchange once data_ops is done

# instantiate permissions with that object
perm = Permissions(session, item)
print(f'Permissions for {item_path}:\n')
print(perm)

### Available permissions on your iRODS server

In [ ]:
perm.available_permissions

### Adding permissions to a collection or data object

In [ ]:
perm.set('modify object', '<username or group name>')
print(perm)

Note that some permission-types have synonyms:

+ read object: 'read', 'read object', 'read_object'
+ modify object: 'write', 'modify object', 'modify_object'

### Removing permissions

In [ ]:
perm.set('null', '<username or group name>')
print(perm)

### Inheritance

Collections have two special permissions level `inherit` and `noinherit`. From the point in time where inheritance in switched on, all newly added subcollections and data objects will inherit their initial permissions from the collection.

In [ ]:
# Retrieve a collection from iRODS
coll_path = session.home
coll = session.irods_session.collections.get(item_path)
coll_perm = Permissions(session, coll)

#Switch inheritance on
coll_perm.set('inherit')
print(coll_perm)

In [ ]:
# Switch inheritance off
coll_perm.set('noinherit')
print(coll_perm)